# How to load data and retrain the model !!!.

In [1]:
# from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime
import pickle
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

Using TensorFlow backend.
C:\Users\Faizan\Desktop\minion_ml_model\env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Faizan\Desktop\minion_ml_model\env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Faizan\Desktop\minion_ml_model\env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Faizan\Desktop

In [5]:
def re_train_model(main_db,*args):
    """
    1. Takes the database as first argument, and any number of CSV file names (without the extension) of new devices as input.
    2. The CSV columns (representing the signature for one minute ) should be as follows:
        A1,PF1,A2,PF2,A3,PF3 ... A59,PF59,A60,PF60
    3. The number of rows should be atleast 100.

    4. Retrains the model. 
    5. Returns the new model.
    """
    #device files are loaded into dataframe and appended to the main database
    devices = [args]
    for x in devices:
        df = pd.read_csv(f'{x}.csv')
        for y in devices:
            if y == x:
                df[f'{y}'] = 1
            else:
                df[f'{y}'] = 0
        main_db = main_db.append(df,ignore_index = True)
    
    
    #creating train,test sets
    col_labels = main_db.columns[120:] 
    X = main_db.drop(labels=col_labels,axis =1)
    y = main_db[col_labels]
    
    
    #determining number of nodes
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    
    
    #defining the model
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
        # fit model
        model.fit(X_train, y_train, verbose=1, epochs=100)
        # make a prediction on the test set
        preds = model.predict(X_test)
        # round probabilities to class labels
        preds = preds.round()
        # calculate accuracy
        acc = accuracy_score(y_test, preds)
        # store result
        print('>%.3f' % acc)
        results.append(acc)
     
        
    print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))
    
    
    prompt = input('input 1 to save the retrained model')
    if prompt == 1:
        date = str(datetime.date.today())
        pickle.dump(model,open(f'minionmodel_{date}.pkl','wb'))
        break
    else:
        print('returning model')
        
    return model